# December 08, 2023

https://adventofcode.com/2023/day/8

In [3]:
import re

In [4]:
test_str = f'''RL

AAA = (BBB, CCC)
BBB = (DDD, EEE)
CCC = (ZZZ, GGG)
DDD = (DDD, DDD)
EEE = (EEE, EEE)
GGG = (GGG, GGG)
ZZZ = (ZZZ, ZZZ)'''

test_str = test_str.split("\n")

In [5]:
test_str2 = f'''LLR

AAA = (BBB, BBB)
BBB = (AAA, ZZZ)
ZZZ = (ZZZ, ZZZ)'''

test_str2 = test_str2.split("\n")

In [1]:
fn = "../data/2023/08.txt"
with open(fn, "r") as file:
    text = file.readlines()

text = [x.strip() for x in text]

In [7]:
def parse_input( text ):
    steps = [0 if x=="L" else 1 for x in text[0] ]

    nodes = dict()
    for line in text[2:]:
        #print(line)
        m = re.fullmatch("(\w\w\w) = \((\w\w\w), (\w\w\w)\)", line)
        #print(m)
        nodes[m[1]] = (m[2], m[3])
    return {'steps':steps, 'nodes':nodes}


In [8]:
test = parse_input(test_str)
test2 = parse_input(test_str2)
puzz = parse_input(text)

### Part 1

In [13]:
def part1( puzz, start='AAA', goal='ZZZ', verbose=False ):
    steps = puzz['steps']
    nodes = puzz['nodes']

    cur = start
    moves = 0
    while cur != goal:
        if verbose:
            print(f'''[{moves}] {cur}''')
        go = steps[ moves % len(steps) ]
        cur = nodes[ cur ][ go ]
        moves += 1

    return moves


In [10]:
part1(test, verbose=True)

[0] AAA
[1] CCC


2

In [14]:
part1(test2, verbose=True)

[0] AAA
[1] BBB
[2] AAA
[3] BBB
[4] AAA
[5] BBB


6

In [15]:
part1(puzz)

20513

### Part 2



In [16]:
test_str3 = f'''LR

11A = (11B, XXX)
11B = (XXX, 11Z)
11Z = (11B, XXX)
22A = (22B, XXX)
22B = (22C, 22C)
22C = (22Z, 22Z)
22Z = (22B, 22B)
XXX = (XXX, XXX)'''

test_str3 = test_str3.split("\n")
test3 = parse_input( test_str3 )

In [17]:
def part2( puzz, max_ghosts=None, verbose=False ):
    steps = puzz['steps']
    nodes = puzz['nodes']

    cur = [ k for k in list(nodes.keys()) if k[-1] == "A" ]
    if max_ghosts is not None and max_ghosts < len(cur):
        cur = cur[:max_ghosts]
    moves = 0

    while not all( [k[-1] == 'Z' for k in cur] ):
        if verbose or moves % 1e6 == 0:
            print(f'''[{moves}] {", ".join(cur)}''')
        go = steps[ moves % len(steps) ]
        cur = [nodes[x][go] for x in cur]
        moves += 1
    
    print(f'''[{moves}] {", ".join(cur)}''')
    return moves

In [18]:
part2( test3, verbose=True )

[0] 11A, 22A
[1] 11B, 22B
[2] 11Z, 22C
[3] 11B, 22Z
[4] 11Z, 22B
[5] 11B, 22C
[6] 11Z, 22Z


6

In [19]:
part2(puzz, 1)

[0] MSA
[14893] XHZ


14893

In [20]:
part2(puzz, 2)

[0] MSA, AAA
[1000000] CNC, JBR
[1087189] XHZ, ZZZ


1087189

In [21]:
# takes too long!!!
#part2(puzz, 3)

In [25]:
def trace_path( puzz, start ):
    nodes = puzz['nodes']
    steps = puzz['steps']

    node_dict = {start: {0:0}}
    cur = start
    moves = 0
    pos = 0

    while True:
        cur = nodes[cur][ steps[pos] ]
        moves += 1
        pos = (pos + 1) % len(steps) # possible wraparound

        if cur in node_dict.keys():
            # we've been here before... see if it's a cycle
            if pos in node_dict[cur].keys():
                break
            node_dict[cur][pos] = moves
        else:
            node_dict[cur] = {pos: moves}
    # end while loop

    finishes = list()
    for node, d in node_dict.items():
        if node[-1] == 'Z':
            finishes += list( d.values() )

    finishes = list(set(finishes))
    cycle = [node_dict[cur][pos], moves]
    return cycle, finishes
            


    

In [23]:
test3

{'steps': [0, 1],
 'nodes': {'11A': ('11B', 'XXX'),
  '11B': ('XXX', '11Z'),
  '11Z': ('11B', 'XXX'),
  '22A': ('22B', 'XXX'),
  '22B': ('22C', '22C'),
  '22C': ('22Z', '22Z'),
  '22Z': ('22B', '22B'),
  'XXX': ('XXX', 'XXX')}}

In [30]:
trace_path( test3, '11A' )

([1, 3], [2])

In [31]:
trace_path( puzz, 'AAA' )

([8, 20521], [20513])

In [33]:
starts = [x for x in puzz['nodes'].keys() if x[-1] == 'A']

cycles = list()
finishes = list()
for s in starts:
    c, f = trace_path( puzz, s )
    cycles.append(c)
    finishes.append(f)
    

In [34]:
cycles

[[2, 14895], [8, 20521], [3, 22202], [2, 19953], [2, 17143], [4, 12087]]

In [35]:
finishes

[[14893], [20513], [22199], [19951], [17141], [12083]]

In [43]:
# fairly simple case since each has only one finish and they're repeatable

ghost_times = [x[0] for x in finishes]
loop_lens = [ x[1]-x[0] for x in cycles]

ghost_times, loop_lens

([14893, 20513, 22199, 19951, 17141, 12083],
 [14893, 20513, 22199, 19951, 17141, 12083])

In [48]:
def find_factors(x):
    fac = [1]
    for i in range(2,x+1):
        if x % i == 0:
            fac.append(i)

    return fac

all_factors = []
for g in ghost_times:
    all_factors.append( find_factors(g) )

In [49]:
all_factors

[[1, 53, 281, 14893],
 [1, 73, 281, 20513],
 [1, 79, 281, 22199],
 [1, 71, 281, 19951],
 [1, 61, 281, 17141],
 [1, 43, 281, 12083]]

In [63]:
# max factor is 281 so gcf is the
x = 53*73*79*71*61*43*281
[x / g for g in ghost_times]

[1074005711.0,
 779757571.0,
 720535477.0,
 801722573.0,
 933152503.0,
 1323774481.0]

In [64]:
x

15995167053923